In [ ]:
import pywt
import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
used_image = 'img/forest.jpg'

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import numpy as np

# Загрузка через PIL
img_pil = Image.open(used_image)

# Загрузка через OpenCV
img_cv = cv2.imread(used_image)
img_cv_rgb = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)  # Конвертация BGR в RGB

# Отображение
plt.figure(figsize=(15, 15))
plt.subplot(1, 2, 1)
plt.imshow(img_pil)
plt.title('PIL Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(img_cv_rgb)
plt.title('OpenCV Image')
plt.axis('off')
plt.show()